In [3]:
# Importing the libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, confusion_matrix
from torch.optim.lr_scheduler import StepLR

In [ ]:
print(torch.torch_version)

In [4]:
# Defining the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Defining the transforms
transform = transforms.Compose([transforms.RandomRotation(30),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

In [6]:
# Loading the dataset
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

In [7]:
# Creating the dataloaders
train_loader = DataLoader(train_data, batch_size=5000, shuffle=True)
test_loader = DataLoader(test_data, batch_size=500, shuffle=True)

In [8]:
# Defining the model
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = self.dropout1(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
# Defining the model
model = Classifier()

In [10]:
# Defining the loss function
criterion = nn.CrossEntropyLoss()

In [11]:
# Defining the optimizer with L2 Regularization
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [12]:
# Defining the number of epochs
epochs = 20

In [ ]:
# Training the model
for epoch in range(epochs):
    train_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    else:
        scheduler.step()
        print(f"Epoch {epoch + 1}, Training loss: {train_loss/len(train_loader)}")

Epoch 1, Training loss: 1.7798164983590443
Epoch 2, Training loss: 1.0188611249128978
Epoch 3, Training loss: 0.7886695166428884
Epoch 4, Training loss: 0.6727325022220612
Epoch 5, Training loss: 0.5864525238672892
Epoch 6, Training loss: 0.543602724870046


In [ ]:
# Testing the model
model.eval()
y_true = []
y_pred = []

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    y_true.append(labels.cpu().numpy())
    y_pred.append(predicted.cpu().numpy())

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

In [ ]:
# Printing the test accuracy
test_acc = accuracy_score(y_true, y_pred)
print(f"Test accuracy: {test_acc*100:.2f}%")

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:")
print(cm)

In [ ]:
# Print the per-class accuracy
class_correct = cm.diagonal()
class_total = cm.sum(axis=1)

for i in range(10):
    if class_total[i] > 0:
        print(f"Accuracy of {i}: {100*class_correct[i]/class_total[i]:.2f}%")
    else:
        print(f"Accuracy of {i}: N/A")